# Palm Print Recognition with Transfer Learning

## Import Required Libraries

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


## Setup Environment and GPU Configuration

In [ ]:
# Setup Environment and GPU Configuration
print(f"TensorFlow Version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU(s) available: {gpus}")
        print("GPU memory growth set.")
    except RuntimeError as e:
        print(f"Error setting up GPU: {e}")
else:
    print("No GPU found, running on CPU.")

## Define Configuration Parameters

In [ ]:
# Configuration parameters for model training
EPOCHS_FEATURE_EXTRACTION = 10
EPOCHS_FINE_TUNE = 5
LEARNING_RATE_INITIAL = 0.001
LEARNING_RATE_FINE_TUNE = 1e-5
BATCH_SIZE = 32
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)

print(f"Training Configuration:")
print(f"- Feature extraction epochs: {EPOCHS_FEATURE_EXTRACTION}")
print(f"- Fine-tuning epochs: {EPOCHS_FINE_TUNE}")
print(f"- Image size: {IMG_SIZE}")
print(f"- Batch size: {BATCH_SIZE}")

## Palm ROI Extraction Function

In [ ]:
def extract_palm_roi(image, size=(128, 128)):
    """
    Extracts a square Region of Interest (ROI) from the center of the palm.
    This is the most critical step for reliable biometric recognition.
    """
    if image is None:
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None

    hand_contour = max(contours, key=cv2.contourArea)
    M = cv2.moments(hand_contour)
    if M["m00"] == 0:
        return None
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])

    roi_size = int(min(image.shape[0], image.shape[1]) * 0.4)
    half_size = roi_size // 2

    x1, y1 = max(0, cx - half_size), max(0, cy - half_size)
    x2, y2 = min(image.shape[1], cx + half_size), min(image.shape[0], cy + half_size)

    roi = image[y1:y2, x1:x2]
    if roi.size == 0:
        return None

    return cv2.resize(roi, size, interpolation=cv2.INTER_AREA)

## Dataset Exploration & Analysis

In [ ]:
import pathlib

# Setup dataset path
DATASET_DIR = 'datasets/Sapienza University Mobile Palmprint Database(SMPD)'
data_dir = pathlib.Path(DATASET_DIR)

print(f"Dataset path set to: {data_dir}")

# Check if dataset exists
if not data_dir.exists():
    print(f"Error: Dataset directory not found at '{DATASET_DIR}'")
    print("Please ensure the path is correct and the data is unzipped.")
else:
    print("Dataset directory found!")

# Find all image files
image_extensions = ['*.jpg', '*.JPG', '*.png', '*.PNG', '*.jpeg', '*.JPEG']
image_paths = []
for ext in image_extensions:
    image_paths.extend(list(data_dir.glob(f'*/{ext}')))

image_count = len(image_paths)
print(f"Total images found: {image_count}")

# Get class names (individual IDs)
class_names_list = sorted([item.name for item in data_dir.glob('*') if item.is_dir()])
num_classes = len(class_names_list)
print(f"Total classes (individuals): {num_classes}")

print("\nSample of class names (IDs):", class_names_list[:10])

# Show image count per class (sample of first 10 classes)
print("\nImage count per class (sample):")
for class_name in class_names_list[:10]:
    class_dir = data_dir / class_name
    count = 0
    for ext in image_extensions:
        count += len(list(class_dir.glob(ext)))
    print(f"- ID {class_name}: {count} images")

## Dataset Loading and Preprocessing

In [ ]:
# Load and Prepare Dataset with ROI Extraction
print("Loading and preprocessing dataset with ROI extraction...")

images = []
labels = []

print(f"Processing {len(class_names_list)} classes...")

for class_idx, class_name in enumerate(class_names_list):
    class_dir = data_dir / class_name
    class_images = []
    
    # Get all image files for this class
    for ext in image_extensions:
        class_images.extend(list(class_dir.glob(ext)))
    
    print(f"Processing class {class_name}: {len(class_images)} images")
    
    for img_path in class_images:
        # Load image
        img = cv2.imread(str(img_path))
        if img is not None:
            # Extract ROI
            roi = extract_palm_roi(img, size=IMG_SIZE)
            if roi is not None:
                images.append(roi)
                labels.append(class_idx)

# Convert to numpy arrays
X = np.array(images)
y = np.array(labels)
class_names = class_names_list

print(f"\nDataset loaded successfully!")
print(f"- Total processed images: {len(X)}")
print(f"- Image shape: {X.shape}")
print(f"- Labels shape: {y.shape}")
print(f"- Number of classes: {len(class_names)}")

## Data Visualization and Analysis

In [ ]:
# Visualize sample processed images
print("--- Visualizing Sample Processed ROI Data ---")
plt.figure(figsize=(15, 10))

# Display first 12 images
for i in range(min(12, len(X))):
    ax = plt.subplot(3, 4, i + 1)
    
    # Convert to displayable format
    img = X[i].astype("uint8")
    plt.imshow(img)
    
    # Get class name from label
    label_idx = y[i]
    class_name = class_names[label_idx]
    plt.title(f"ID: {class_name}")
    plt.axis("off")

plt.suptitle("Sample Palm ROI Images from Dataset", fontsize=16)
plt.tight_layout()
plt.show()

# Show class distribution analysis
print("\n--- Class Distribution Analysis ---")
unique_labels, counts = np.unique(y, return_counts=True)
print(f"Images per class - Min: {counts.min()}, Max: {counts.max()}, Mean: {counts.mean():.1f}")

# Plot class distribution (for first 20 classes if there are many)
plt.figure(figsize=(12, 6))
classes_to_show = min(20, len(unique_labels))
plt.bar(range(classes_to_show), counts[:classes_to_show])
plt.title(f"Images per Class (First {classes_to_show} classes)")
plt.xlabel("Class Index")
plt.ylabel("Number of Images")
plt.xticks(range(classes_to_show), [class_names[i] for i in range(classes_to_show)], rotation=45)
plt.tight_layout()
plt.show()

## Train-Validation Split

In [ ]:
# Train-Validation Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Dataset split:")
print(f"- Training samples: {len(X_train)}")
print(f"- Validation samples: {len(X_val)}")
print(f"- Training shape: {X_train.shape}")
print(f"- Validation shape: {X_val.shape}")

## Create TensorFlow Datasets

In [ ]:
# Create TensorFlow datasets with optimization
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

print(f"TensorFlow datasets created:")
print(f"- Training batches: {len(train_ds)}")
print(f"- Validation batches: {len(val_ds)}")
print(f"- Batch size: {BATCH_SIZE}")
print(f"- Image size: {IMG_SIZE}")

## Data Augmentation Setup

In [ ]:
# Data augmentation layers for training
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.2),
], name='augmentation')

print("--- Preview of Data Augmentation ---")

# Take one batch and show original vs augmented versions
for images, labels in train_ds.take(1):
    sample_image = images[0:1]  # Take first image
    sample_label = labels[0]
    
    plt.figure(figsize=(15, 3))
    
    # Original image
    plt.subplot(1, 5, 1)
    plt.imshow(sample_image[0].numpy().astype("uint8"))
    plt.title(f"Original\nID: {class_names[sample_label]}")
    plt.axis("off")
    
    # Show 4 augmented versions
    for i in range(4):
        augmented = data_augmentation(sample_image)
        plt.subplot(1, 5, i + 2)
        plt.imshow(augmented[0].numpy().astype("uint8"))
        plt.title(f"Augmented {i+1}")
        plt.axis("off")
    
    plt.suptitle("Data Augmentation Preview", fontsize=14)
    plt.tight_layout()
    plt.show()
    break

print("Data augmentation setup complete!")

## Build Base Model Architecture

In [ ]:
# Build Model with Transfer Learning using MobileNetV2
print("--- Building Model using Transfer Learning (MobileNetV2) ---")
IMG_SHAPE = IMG_SIZE + (3,)

# Load pre-trained MobileNetV2 base model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)
base_model._name = 'mobilenetv2_base'
base_model.trainable = False  # Freeze base model initially

print(f"Base model loaded: {base_model.name}")
print(f"Base model has {len(base_model.layers)} layers")
print(f"Input shape: {IMG_SHAPE}")
print("Base model is frozen for initial training")

In [ ]:
# Build complete model architecture
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = layers.Rescaling(1./255)(inputs)
x = data_augmentation(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

print("Model architecture built successfully!")
print(f"Model will classify {len(class_names)} classes")
print(f"Final dense layer output: {len(class_names)} neurons")

## Compile Model for Feature Extraction

In [ ]:
# Compile Model for Feature Extraction
print("--- Compiling the Model ---")
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_INITIAL),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Setup callbacks for training
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        'best_palm_model.keras', 
        save_best_only=True, 
        monitor='val_accuracy', 
        mode='max',
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        patience=5, 
        monitor='val_loss', 
        restore_best_weights=True,
        verbose=1
    )
]

print("Model compiled and callbacks configured!")
print(f"Optimizer: Adam with learning rate {LEARNING_RATE_INITIAL}")
print("Loss function: sparse_categorical_crossentropy")
print("Metrics: accuracy")

## Phase 1 Training - Feature Extraction

In [ ]:
# Phase 1 Training - Feature Extraction
print("--- Starting Training Phase 1: Feature Extraction ---")
print("Base model is frozen, training only the classifier head...")

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FEATURE_EXTRACTION,
    callbacks=callbacks,
    verbose=1
)

print("--- Phase 1 Training Complete ---")
print(f"Completed {len(history.history['accuracy'])} epochs")
print(f"Final training accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")

## Model Fine-tuning Setup

In [ ]:
# Model Fine-tuning Setup
print("--- Preparing for Fine-Tuning ---")

# Unfreeze base model for fine-tuning
base_model = model.get_layer('mobilenetv2_base')
base_model.trainable = True

# Freeze early layers, only fine-tune top layers
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print(f"Fine-tuning from layer {fine_tune_at} onwards...")
print(f"Total layers in base model: {len(base_model.layers)}")
print(f"Trainable layers: {len([layer for layer in base_model.layers if layer.trainable])}")

# Recompile with lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_FINE_TUNE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Model recompiled with learning rate: {LEARNING_RATE_FINE_TUNE}")
print("Ready for fine-tuning phase!")

## Phase 2 Training - Fine Tuning

In [ ]:
# Phase 2 Training - Fine Tuning
print("--- Starting Training Phase 2: Fine-Tuning ---")

# Continue training with fine-tuning
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FEATURE_EXTRACTION + EPOCHS_FINE_TUNE,
    initial_epoch=history.epoch[-1] + 1,
    callbacks=callbacks,
    verbose=1
)

print("--- Phase 2 Fine-Tuning Complete ---")
print(f"Completed {len(history_fine.history['accuracy'])} additional epochs")
print(f"Final training accuracy: {history_fine.history['accuracy'][-1]:.4f}")
print(f"Final validation accuracy: {history_fine.history['val_accuracy'][-1]:.4f}")

## Model Evaluation and Results

In [ ]:
# Model Evaluation and Results
print("--- Final Model Evaluation ---")

# Load best model
best_model = tf.keras.models.load_model('best_palm_model.keras')

# Evaluate on validation set
loss, accuracy = best_model.evaluate(val_ds, verbose=1)
print(f"\nFinal Model Performance:")
print(f"- Validation Loss: {loss:.4f}")
print(f"- Validation Accuracy: {accuracy * 100:.2f}%")
print(f"- Total classes: {len(class_names)}")
print(f"- Training samples: {len(X_train)}")
print(f"- Validation samples: {len(X_val)}")

## Training History Visualization

In [ ]:
# Training History Visualization
print("--- Plotting Training History ---")

# Combine training histories from both phases
acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']
loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.axvline(len(history.history['accuracy']) - 1, color='gray', linestyle='--', alpha=0.7, label='Start Fine-Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.axvline(len(history.history['loss']) - 1, color='gray', linestyle='--', alpha=0.7, label='Start Fine-Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("Training history visualization complete!")
print("Plot saved to: 'training_history.png'")

## Save Final Model

In [ ]:
# Save Final Model and Summary
print("--- Training Complete! ---")
print("\nModel and Results Summary:")
print(f"✓ Model saved to: 'best_palm_model.keras'")
print(f"✓ Training history plot saved to: 'training_history.png'")
print(f"✓ Total classes trained: {len(class_names)}")
print(f"✓ Total training images: {len(X_train)}")
print(f"✓ Total validation images: {len(X_val)}")
print(f"✓ Image size used: {IMG_SIZE}")
print(f"✓ Batch size: {BATCH_SIZE}")
print(f"✓ Feature extraction epochs: {EPOCHS_FEATURE_EXTRACTION}")
print(f"✓ Fine-tuning epochs: {EPOCHS_FINE_TUNE}")
print(f"✓ Final validation accuracy: {accuracy * 100:.2f}%")

print("\nModel is ready for inference!")
print("You can load the model using: tf.keras.models.load_model('best_palm_model.keras')")